# Evaluation on Turk Crowdsourced Aspect-Opinion Pair (3) Manual KP Multi-Aspect

In [18]:
import json
import pandas as pd
import numpy as np
# import spacy
import matplotlib.pyplot as plt
import ast

pd.set_option('display.max_colwidth', None)

In [2]:
from datasets import concatenate_datasets, load_dataset
from datasets import Dataset, DatasetDict

In [4]:
from utils.track_1_kp_matching import *

In [5]:
from sentence_transformers import SentenceTransformer, InputExample, LoggingHandler, losses, models, util
import torch

In [6]:
from datasets import concatenate_datasets, load_dataset
from datasets import Dataset, DatasetDict
from torch.utils.data import DataLoader

First with top 10 args per category labelling of test set

## Load Test dataset

In [7]:
df = pd.read_pickle(f"../data/Evaluation/test_data.pkl")

In [8]:
# df = pd.read_pickle("../../../ABSA Dataset/MTURK/snippext_crowdsourced_multi_aspect_6_15_sample_top_8_arg_5000_pairs_manual_kps_arg_8_annotators_2.pkl")

In [9]:
# df = pd.read_pickle("../../../ABSA Dataset/MTURK/snippext_crowdsourced_multi_aspect_6_15_sample_top_8_arg_5000_pairs_manual_kps_arg_8_annotators_2_0.5.pkl")

In [10]:
# df = pd.read_pickle("../../../ABSA Dataset/MTURK/snippext_crowdsourced_multi_aspect_6_15_sample_top_8_arg_5000_pairs_manual_kps_arg_8_annotators_2_0.6.pkl")

In [11]:
df

,HITId,comment_id,label,full_key_point,key_point_id,comment,full_comment,attributes,topic,isMultiAspect,...,old_key_point_id,key_point,aspects_x,opinions_x,opinion_aspect_pairs_x,sentiments_x,aspects_y,opinions_y,opinion_aspect_pairs_y,sentiments_y
0,302U8RURJZ1WH0D09XZ8R7HSOP5NVW,arg_0_537,1,Terrible administration and management .,kp_0_2,"[uninviting experience <SEP> negative, rude employees <SEP> negative]",Employees are rude and make the experience uninviting .,"[restaurant -> atmosphere, staff]",Arts & Entertainment,True,...,kp_0_2,terrible administration <SEP> negative,"[experience, employees]","[uninviting, rude]","[uninviting experience, rude employees]","[negative, negative]",administration,terrible,['terrible administration'],['negative']
1,302U8RURJZ1WH0D09XZ8R7HSOP5NVW,arg_0_537,1,Rude and Unprofessional staff .,kp_0_24,"[uninviting experience <SEP> negative, rude employees <SEP> negative]",Employees are rude and make the experience uninviting .,"[restaurant -> atmosphere, staff]",Arts & Entertainment,True,...,kp_0_24,unprofessional staff <SEP> negative,"[experience, employees]","[uninviting, rude]","[uninviting experience, rude employees]","[negative, negative]",staff,unprofessional,['unprofessional staff'],['negative']
2,302U8RURJZ1WH0D09XZ8R7HSOP5NVW,arg_0_537,0,Great for conventions and concerts ! !,kp_0_4,"[uninviting experience <SEP> negative, rude employees <SEP> negative]",Employees are rude and make the experience uninviting .,"[restaurant -> atmosphere, staff]",Arts & Entertainment,True,...,kp_0_4,great for conventions <SEP> positive,"[experience, employees]","[uninviting, rude]","[uninviting experience, rude employees]","[negative, negative]",conventions,great for,['great for conventions'],['positive']
3,302U8RURJZ1WH0D09XZ8R7HSOP5NVW,arg_0_537,0,Horrible customer service .,kp_0_5,"[uninviting experience <SEP> negative, rude employees <SEP> negative]",Employees are rude and make the experience uninviting .,"[restaurant -> atmosphere, staff]",Arts & Entertainment,True,...,kp_0_5,horrible customer service <SEP> negative,"[experience, employees]","[uninviting, rude]","[uninviting experience, rude employees]","[negative, negative]",customer service,horrible,['horrible customer service'],['negative']
4,302U8RURJZ1WH0D09XZ8R7HSOP5NVW,arg_0_537,0,It had wonderful acoustics .,kp_0_0,"[uninviting experience <SEP> negative, rude employees <SEP> negative]",Employees are rude and make the experience uninviting .,"[restaurant -> atmosphere, staff]",Arts & Entertainment,True,...,kp_0_0,wonderful acoustics <SEP> positive,"[experience, employees]","[uninviting, rude]","[uninviting experience, rude employees]","[negative, negative]",acoustics,wonderful,['wonderful acoustics'],['positive']
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6794,3ZXV7Q5FJBOOUZUJKSLQTAKU1UIFC3,arg_4_6440,0,Great family owned restaurant .,kp_4_25,[friendly staff <SEP> positive],The staff are friendly and attentive .,[staff],Restaurants,False,...,kp_4_25,great family owned restaurant <SEP> positive,[staff],[friendly],[friendly staff],[positive],restaurant,great family owned,['great family owned restaurant'],['positive']
6795,3ZXV7Q5FJBOOUZUJKSLQTAKU1UIFC3,arg_4_6440,0,Worst service in the world !,kp_4_27,[friendly staff <SEP> positive],The staff are friendly and attentive .,[staff],Restaurants,False,...,kp_4_27,worst service <SEP> negative,[staff],[friendly],[friendly staff],[positive],service,worst,['worst service'],['negative']
6796,3ZXV7Q5FJBOOUZUJKSLQTAKU1UIFC3,arg_4_6440,0,Rude and inattentive staff .,kp_4_30,[friendly staff <SEP> positive],The staff are friendly and attentive .,[staff],Restaurants,False,...,kp_4_30,inattentive staff <SEP> negative,[staff],[friendly],[friendly staff],[positive],staff,inattentive,['inattentive staff'],['negative']
6797,3ZXV7Q5FJBOOUZUJKSLQTAKU1UIFC3,arg_4_6440,0,Great sushi and sashimi .,kp_4_31,[friendly staff <SEP> positive],The staff are friendly and attentive .,[st

In [12]:
df['label'].value_counts()

0    6380
1     419
Name: label, dtype: int64

In [13]:
# df = df[df['topic'] == 'restaurant']

In [14]:
# # df = df[df['topic'] == 'Hotels']
# # df = df[df['topic'] == 'Restaurants']
# # df = df[df['topic'] == 'Arts & Entertainment']
# df = df[df['topic'] == 'Automotive']
# # df = df[df['topic'] == 'Beauty & Spas']

In [15]:
# df = df[df['isMultiAspect'] == True]

In [16]:
def prepare_comment_kp_label_input(df):
    df['num_of_aspects'] = df['comment'].str.len()
    df = df.rename(columns={'comment_id': 'comment_id_sent'})
    df['comment_id'] = df['comment_id_sent'] + "_" + df['num_of_aspects'].astype(str)
    
    comment_df = df[['comment_id', 'topic', 'full_comment', 'num_of_aspects',
                'aspects_x', 'opinions_x', 'opinion_aspect_pairs_x', 'sentiments_x', 'attributes']]
    comment_df = comment_df.drop_duplicates(subset=['comment_id']).reset_index(drop=True)
    comment_df.columns = [col.replace("_x", "") for col in comment_df.columns]
    
    kp_df = df[['key_point_id', 'topic', 'full_key_point',
           'aspects_y', 'opinions_y', 'opinion_aspect_pairs_y', 'sentiments_y', 'attributes']]
    kp_df = kp_df.drop_duplicates(subset=['key_point_id']).reset_index(drop=True)
    kp_df = kp_df.rename(columns={'full_key_point': 'key_point'})
    kp_df.columns = [col.replace("_y", "") for col in kp_df.columns]
    kp_df['opinion_aspect_pairs'] = kp_df['opinion_aspect_pairs'].apply(lambda x: ast.literal_eval(x))
    kp_df['sentiments'] = kp_df['sentiments'].apply(lambda x: ast.literal_eval(x))
    
    labels_df = df[['comment_id', 'key_point_id', 'label']]
    
    return comment_df, kp_df, labels_df

In [31]:
comment_df, kp_df, labels_df = prepare_comment_kp_label_input(df)

## Merging

In [32]:
comment_df = comment_df.explode(['aspects', 'opinions', 'opinion_aspect_pairs', 'sentiments', 'attributes'])

In [33]:
comment_df

,comment_id,topic,full_comment,num_of_aspects,aspects,opinions,opinion_aspect_pairs,sentiments,attributes
0,arg_0_537_2,Arts & Entertainment,Employees are rude and make the experience uninviting .,2,experience,uninviting,uninviting experience,negative,restaurant -> atmosphere
0,arg_0_537_2,Arts & Entertainment,Employees are rude and make the experience uninviting .,2,employees,rude,rude employees,negative,staff
1,arg_2_1872_2,Beauty & Spas,She is not only an excellent hairstylist but a good conversationalist .,2,hairstylist,excellent,excellent hairstylist,positive,food -> quality
1,arg_2_1872_2,Beauty & Spas,She is not only an excellent hairstylist but a good conversationalist .,2,conversationalist,good,good conversationalist,positive,restaurant -> atmosphere
2,arg_1_1148_2,Automotive,"Terrible job , overpriced for zero quality .",2,job,terrible,terrible job,negative,staff
...,...,...,...,...,...,...,...,...,...
235,arg_2_1459_1,Beauty & Spas,Mark provides world class customer service !,1,customer service,world class,world class customer service,positive,staff
236,arg_2_1933_1,Beauty & Spas,Staff is well versed in nail tech techniques and services .,1,staff,well versed in,well versed in staff,positive,staff
237,arg_1_249_1,Automotive,Everybody is so rude & disrespectful !,1,everybody,rude,rude everybody,negative,staff
238,arg_3_3011_1,Hotels,"The bathrooms are clean , spacious , and well appointed .",1,bathrooms,clean,clean bathrooms,positive,restaurant -> atmosphere


In [34]:
kp_df

,key_point_id,topic,key_point,aspects,opinions,opinion_aspect_pairs,sentiments,attributes
0,kp_0_2,Arts & Entertainment,Terrible administration and management .,administration,terrible,[terrible administration],[negative],"[restaurant -> atmosphere, staff]"
1,kp_0_24,Arts & Entertainment,Rude and Unprofessional staff .,staff,unprofessional,[unprofessional staff],[negative],"[restaurant -> atmosphere, staff]"
2,kp_0_4,Arts & Entertainment,Great for conventions and concerts ! !,conventions,great for,[great for conventions],[positive],"[restaurant -> atmosphere, staff]"
3,kp_0_5,Arts & Entertainment,Horrible customer service .,customer service,horrible,[horrible customer service],[negative],"[restaurant -> atmosphere, staff]"
4,kp_0_0,Arts & Entertainment,It had wonderful acoustics .,acoustics,wonderful,[wonderful acoustics],[positive],"[restaurant -> atmosphere, staff]"
...,...,...,...,...,...,...,...,...
135,kp_3_30,Hotels,Free washer and dryer,washer,free,[free washer],[positive],[staff]
136,kp_3_31,Hotels,Shower had solid water pressure .,water pressure,solid,[solid water pressure],[positive],[staff]
137,kp_3_32,Hotels,Wifi was very reliable .,wifi,very reliable,[very reliable wifi],[positive],[staff]
138,kp_3_33,Hotels,Prices were fair and reasonable .,prices,fair and reasonable,[fair and reasonable prices],[positive],[staff]


In [35]:
kp_df = kp_df.explode(['opinion_aspect_pairs', 'sentiments'])
kp_df

,key_point_id,topic,key_point,aspects,opinions,opinion_aspect_pairs,sentiments,attributes
0,kp_0_2,Arts & Entertainment,Terrible administration and management .,administration,terrible,terrible administration,negative,"[restaurant -> atmosphere, staff]"
1,kp_0_24,Arts & Entertainment,Rude and Unprofessional staff .,staff,unprofessional,unprofessional staff,negative,"[restaurant -> atmosphere, staff]"
2,kp_0_4,Arts & Entertainment,Great for conventions and concerts ! !,conventions,great for,great for conventions,positive,"[restaurant -> atmosphere, staff]"
3,kp_0_5,Arts & Entertainment,Horrible customer service .,customer service,horrible,horrible customer service,negative,"[restaurant -> atmosphere, staff]"
4,kp_0_0,Arts & Entertainment,It had wonderful acoustics .,acoustics,wonderful,wonderful acoustics,positive,"[restaurant -> atmosphere, staff]"
...,...,...,...,...,...,...,...,...
135,kp_3_30,Hotels,Free washer and dryer,washer,free,free washer,positive,[staff]
136,kp_3_31,Hotels,Shower had solid water pressure .,water pressure,solid,solid water pressure,positive,[staff]
137,kp_3_32,Hotels,Wifi was very reliable .,wifi,very reliable,very reliable wifi,positive,[staff]
138,kp_3_33,Hotels,Prices were fair and reasonable .,prices,fair and reasonable,fair and reasonable prices,positive,[staff]


In [37]:
# Should not include attribute during matching to not favouring our apporach during evaluation, because other baselines don't
# use attribute as well during the matching
sent_kp_df = comment_df.astype(str).merge(kp_df.astype(str), how='left', on=['topic']).dropna(subset=['key_point']) 
sent_kp_df = sent_kp_df.reset_index(drop=True)
sent_kp_df

,comment_id,topic,full_comment,num_of_aspects,aspects_x,opinions_x,opinion_aspect_pairs_x,sentiments_x,attributes_x,key_point_id,key_point,aspects_y,opinions_y,opinion_aspect_pairs_y,sentiments_y,attributes_y
0,arg_0_537_2,Arts & Entertainment,Employees are rude and make the experience uninviting .,2,experience,uninviting,uninviting experience,negative,restaurant -> atmosphere,kp_0_2,Terrible administration and management .,administration,terrible,terrible administration,negative,"['restaurant -> atmosphere', 'staff']"
1,arg_0_537_2,Arts & Entertainment,Employees are rude and make the experience uninviting .,2,experience,uninviting,uninviting experience,negative,restaurant -> atmosphere,kp_0_24,Rude and Unprofessional staff .,staff,unprofessional,unprofessional staff,negative,"['restaurant -> atmosphere', 'staff']"
2,arg_0_537_2,Arts & Entertainment,Employees are rude and make the experience uninviting .,2,experience,uninviting,uninviting experience,negative,restaurant -> atmosphere,kp_0_4,Great for conventions and concerts ! !,conventions,great for,great for conventions,positive,"['restaurant -> atmosphere', 'staff']"
3,arg_0_537_2,Arts & Entertainment,Employees are rude and make the experience uninviting .,2,experience,uninviting,uninviting experience,negative,restaurant -> atmosphere,kp_0_5,Horrible customer service .,customer service,horrible,horrible customer service,negative,"['restaurant -> atmosphere', 'staff']"
4,arg_0_537_2,Arts & Entertainment,Employees are rude and make the experience uninviting .,2,experience,uninviting,uninviting experience,negative,restaurant -> atmosphere,kp_0_0,It had wonderful acoustics .,acoustics,wonderful,wonderful acoustics,positive,"['restaurant -> atmosphere', 'staff']"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8587,arg_4_6440_1,Restaurants,The staff are friendly and attentive .,1,staff,friendly,friendly staff,positive,staff,kp_4_28,The service here was exceptional .,service,exceptional,exceptional service,positive,"['restaurant -> comfort', 'food -> quality']"
8588,arg_4_6440_1,Restaurants,The staff are friendly and attentive .,1,staff,friendly,friendly staff,positive,staff,kp_4_29,Staff was courteous and accommodating .,staff,courteous,courteous staff,positive,"['restaurant -> comfort', 'food -> quality']"
8589,arg_4_6440_1,Restaurants,The staff are friendly and attentive .,1,staff,friendly,friendly staff,positive,staff,kp_4_30,Rude and inattentive staff .,staff,inattentive,inattentive staff,negative,"['restaurant -> comfort', 'food -> quality']"
8590,arg_4_6440_1,Restaurants,The staff are friendly and attentive .,1,staff,friendly,friendly staff,positive,staff,kp_4_31,Great sushi and sashimi .,sushi,great,great sushi,positive,"['restaurant -> comfort', 'food -> quality']"


In [38]:
# sent_kp_df = arg_df.astype(str).merge(kp_df.astype(str), how='left', on=['topic', 'attributes']).dropna(subset=['key_point'])
# sent_kp_df = sent_kp_df.reset_index(drop=True)
# sent_kp_df

In [39]:
# sent_kp_df.drop_duplicates(subset=['opinion_aspect_pairs_x', 'opinion_aspect_pairs_y'])

In [40]:
# sent_kp_df[sent_kp_df['opinion_aspect_pairs_x'] == sent_kp_df['opinion_aspect_pairs_y']]

In [41]:
sent_kp_df.loc[
    (sent_kp_df['aspects_x'] == sent_kp_df['aspects_y']) & (sent_kp_df['sentiments_x'] == sent_kp_df['sentiments_y']),
    'score'
] = 1

In [42]:
sent_kp_df.loc[
    (sent_kp_df['sentiments_x'] != sent_kp_df['sentiments_y']),
    'score'
] = 0

In [43]:
sent_kp_df[sent_kp_df['score'] == 0]

,comment_id,topic,full_comment,num_of_aspects,aspects_x,opinions_x,opinion_aspect_pairs_x,sentiments_x,attributes_x,key_point_id,key_point,aspects_y,opinions_y,opinion_aspect_pairs_y,sentiments_y,attributes_y,score
2,arg_0_537_2,Arts & Entertainment,Employees are rude and make the experience uninviting .,2,experience,uninviting,uninviting experience,negative,restaurant -> atmosphere,kp_0_4,Great for conventions and concerts ! !,conventions,great for,great for conventions,positive,"['restaurant -> atmosphere', 'staff']",0.0
4,arg_0_537_2,Arts & Entertainment,Employees are rude and make the experience uninviting .,2,experience,uninviting,uninviting experience,negative,restaurant -> atmosphere,kp_0_0,It had wonderful acoustics .,acoustics,wonderful,wonderful acoustics,positive,"['restaurant -> atmosphere', 'staff']",0.0
7,arg_0_537_2,Arts & Entertainment,Employees are rude and make the experience uninviting .,2,experience,uninviting,uninviting experience,negative,restaurant -> atmosphere,kp_0_6,Superior customer service !,customer service,superior,superior customer service,positive,"['restaurant -> atmosphere', 'staff']",0.0
9,arg_0_537_2,Arts & Entertainment,Employees are rude and make the experience uninviting .,2,experience,uninviting,uninviting experience,negative,restaurant -> atmosphere,kp_0_8,cover and reasonably priced drinks .,drinks,reasonably priced,reasonably priced drinks,positive,"['restaurant -> atmosphere', 'staff']",0.0
10,arg_0_537_2,Arts & Entertainment,Employees are rude and make the experience uninviting .,2,experience,uninviting,uninviting experience,negative,restaurant -> atmosphere,kp_0_9,Drinks were delicious and strong .,drinks,delicious,delicious drinks,positive,"['restaurant -> atmosphere', 'staff']",0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8581,arg_4_6440_1,Restaurants,The staff are friendly and attentive .,1,staff,friendly,friendly staff,positive,staff,kp_4_22,Way over priced .,priced,over,over priced,negative,"['restaurant -> comfort', 'food -> quality']",0.0
8583,arg_4_6440_1,Restaurants,The staff are friendly and attentive .,1,staff,friendly,friendly staff,positive,staff,kp_4_24,The quality has diminished significantly .,quality,diminished significantly,diminished significantly quality,negative,"['restaurant -> comfort', 'food -> quality']",0.0
8586,arg_4_6440_1,Restaurants,The staff are friendly and attentive .,1,staff,friendly,friendly staff,positive,staff,kp_4_27,Worst service in the world !,service,worst,worst service,negative,"['restaurant -> comfort', 'food -> quality']",0.0
8589,arg_4_6440_1,Restaurants,The staff are friendly and attentive .,1,staff,friendly,friendly staff,positive,staff,kp_4_30,Rude and inattentive staff .,staff,inattentive,inattentive staff,negative,"['restaurant -> comfort', 'food -> quality']",0.0


Handle implicit match

In [44]:
# import spacy

# nlp = spacy.load('en_core_web_md')

# print("Enter two space-separated words")
# words = input()

# tokens = nlp(words)

# for token in tokens:
#     # Printing the following attributes of each token.
#     # text: the word string, has_vector: if it contains
#     # a vector representation in the model,
#     # vector_norm: the algebraic norm of the vector,
#     # is_oov: if the word is out of vocabulary.
#     print(token.text, token.has_vector, token.vector_norm, token.is_oov)

# token1, token2 = tokens[0], tokens[1:]

# print("Similarity:", token1.similarity(token2))

In [45]:
import spacy

nlp = spacy.load('en_core_web_md')

2023-07-12 10:52:49.294991: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
2023-07-12 10:52:50.234145: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcuda.so.1
2023-07-12 10:52:50.234535: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:923] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-07-12 10:52:50.234698: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: NVIDIA GeForce RTX 3080 Ti computeCapability: 8.6
coreClock: 1.77GHz coreCount: 80 deviceMemorySize: 12.00GiB deviceMemoryBandwidth: 849.46GiB/s
2023-07-12 10:52:50.234767: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
2023-07-12 10:52:50.234803: I tensorflow/stream_ex

In [46]:
# import spacy

# nlp = spacy.load('en_core_web_md')

# print("Enter two space-separated words")
# words = input()

# tokens = nlp(words)

# for token in tokens:
#     # Printing the following attributes of each token.
#     # text: the word string, has_vector: if it contains
#     # a vector representation in the model,
#     # vector_norm: the algebraic norm of the vector,
#     # is_oov: if the word is out of vocabulary.
#     print(token.text, token.has_vector, token.vector_norm, token.is_oov)

# sep_index = [token.i for token in tokens if token.text == '|'][0]
    
# # token1, token2 = tokens[0], tokens[1:]

# # print("Similarity:", token1.similarity(token2))

In [47]:
# token1, token2 = tokens[:sep_index], tokens[sep_index+1:]
# token1.similarity(token2)

In [48]:
# token2.similarity(token1)

In [49]:
import warnings
warnings.filterwarnings("error")

In [50]:
sent_kp_df['predicted_by_cosine'] = False

In [51]:
unfound_tokens = []
def calculate_aspects_semantic_similarity(row):
    tokens = nlp(row['aspects_x'] + " | " + row['aspects_y'])
    sep_index = [token.i for token in tokens if token.text == '|'][0]
    token1, token2 = tokens[:sep_index], tokens[sep_index+1:]
    row['aspects_x_len'] = len(token1)
    row['aspects_y_len'] = len(token2)
#     print(token1, token2)
    try:
        print(token2.similarity(token1))
        row['score'] = token2.similarity(token1)
    except:
#         display(row)
#         unfound_tokens += [token1]
        row['score'] = 0
    row['predicted_by_cosine'] = True
    
    return row

In [52]:
# implicit_match_analyze_mask = sent_kp_df['aspects_x'] != sent_kp_df['aspects_y']
implicit_match_analyze_mask = pd.isnull(sent_kp_df['score'])
sent_kp_df[implicit_match_analyze_mask] = sent_kp_df[implicit_match_analyze_mask].apply(calculate_aspects_semantic_similarity, axis=1)

0.479239821434021
0.404928058385849
0.6046743392944336
0.3221808671951294
0.26104822754859924
0.20260260999202728
0.3452816903591156
0.38468843698501587
0.24648836255073547
0.20801319181919098
0.5071344375610352
0.7234411835670471
0.6231473088264465
0.03593863174319267
0.22788509726524353
0.12307243794202805
0.39873942732810974
0.27384090423583984
0.1558079868555069
0.23306694626808167
-0.012764311395585537
0.021436000242829323
-0.03670242801308632
0.1575903296470642
0.012230013497173786
0.19626007974147797
0.0423983633518219
0.23727945983409882
0.3592225909233093
0.25248175859451294
0.03425845131278038
0.10977999866008759
0.4419323801994324
-0.012764311395585537
0.09814277291297913
0.46655479073524475
0.49952998757362366
0.3213580548763275
0.46941202878952026
0.3134591579437256
0.06363898515701294
0.5443854331970215
0.14920848608016968
0.13292981684207916
0.2670433521270752
0.4891284108161926
0.3520849645137787
0.022836240008473396
0.46655479073524475
0.20959238708019257
0.09381703287

0.3737960457801819
0.49863967299461365
0.2534477114677429
0.2534477114677429
0.23999233543872833
0.38107115030288696
0.2813397943973541
0.2813397943973541
0.2813397943973541
0.18440529704093933
0.26019880175590515
-0.026973821222782135
0.04775035008788109
0.14596562087535858
0.13867683708667755
0.5920196771621704
0.5303775072097778
0.533624529838562
0.6232120394706726
0.2813397943973541
0.2813397943973541
0.1975843906402588
0.3655531108379364
0.16401387751102448
0.39057785272598267
0.6195873022079468
0.3766213059425354
0.22712752223014832
0.5520296692848206
0.5520296692848206
0.4461071193218231
0.004728779196739197
0.17304359376430511
0.17304359376430511
0.17304359376430511
0.19446949660778046
0.16946181654930115
0.04161803051829338
0.04279543459415436
0.1506892293691635
0.15682989358901978
0.6095215082168579
0.3026359975337982
0.37787145376205444
0.46647903323173523
0.17304359376430511
0.17304359376430511
0.148757666349411
0.397859662771225
0.11306819319725037
0.2540786564350128
0.421

0.1762857735157013
0.32784396409988403
0.32784396409988403
0.366007924079895
0.3111179769039154
0.01059881690889597
0.19629646837711334
0.3613211214542389
0.1218675822019577
0.2709222733974457
0.0018994399579241872
0.036234498023986816
-0.02703549899160862
0.08088632673025131
0.29820001125335693
0.15075138211250305
0.2165852040052414
0.2717789113521576
0.1963033676147461
0.3092896342277527
0.05601758509874344
0.15375757217407227
0.15375757217407227
0.15494655072689056
0.22336989641189575
0.07188506424427032
0.2547210156917572
0.07056597620248795
0.10419237613677979
0.1853417605161667
0.3614841401576996
0.3614841401576996
0.3614841401576996
0.4572821259498596
0.5073227882385254
0.2094154804944992
0.21663276851177216
0.18761296570301056
0.2981131076812744
0.287257581949234
0.44231700897216797
0.23756329715251923
0.6241921186447144
0.3614841401576996
0.3614841401576996
0.3627637028694153
0.259442538022995
0.3719464838504791
0.4987111985683441
0.40393251180648804
0.254658579826355
0.394114

0.39208561182022095
0.26180434226989746
0.22770267724990845
-0.001756626064889133
0.29488858580589294
0.07713549584150314
0.13963022828102112
0.2477026879787445
0.019598454236984253
0.019598454236984253
0.04744689166545868
0.2699221074581146
0.6231473088264465
0.27384090423583984
0.17282633483409882
0.314348429441452
0.5311083197593689
0.7234411835670471
0.2946145832538605
0.5239914059638977
0.5005245208740234
0.22745375335216522
0.26115840673446655
0.352783203125
0.2221427857875824
0.21083298325538635
0.528118908405304
0.16649234294891357
0.27308398485183716
0.3234666883945465
0.3234666883945465
0.2574889063835144
0.3630084991455078
0.19392134249210358
0.04325667768716812
0.174994558095932
0.11408208310604095
0.17016199231147766
0.4170213043689728
0.1985890120267868
0.4374772310256958
0.3968614637851715
0.38454046845436096
0.5149292945861816
0.35422638058662415
0.5295965671539307
0.23763880133628845
0.3936721682548523
0.22894468903541565
0.48307037353515625
0.16022354364395142
0.37315

0.3968614637851715
0.42208072543144226
0.21646293997764587
0.5560011863708496
1.0000001192092896
0.24556170403957367
0.8065881133079529
0.07415267080068588
0.47657689452171326
-0.014009324833750725
0.059809111058712006
0.3968614637851715
0.3968614637851715
0.16451874375343323
0.28028497099876404
0.09125599265098572
0.18662913143634796
0.023703889921307564
0.12182255834341049
0.32157981395721436
0.04043014720082283
0.04043014720082283
0.009173517115414143
0.38582196831703186
0.4793057441711426
0.4317000210285187
0.47723153233528137
0.04256531596183777
0.04256531596183777
0.25185292959213257
0.31519022583961487
0.22921694815158844
0.6272557973861694
0.3750530481338501
0.10563243925571442
0.22374531626701355
0.2769460082054138
0.05270698294043541
0.3309778571128845
0.132156103849411
0.39924779534339905
0.19358426332473755
0.28231072425842285
0.267300009727478
0.259655237197876
0.37622958421707153
0.4096202850341797
0.8972026705741882
0.5311083197593689
0.3620019853115082
0.500524520874023

...

0.11408208310604095
0.17016199231147766
0.33772003650665283
0.7088468074798584
0.5080896615982056
0.25735345482826233
0.7390003204345703
0.7390003204345703
0.6372321248054504
0.6372321248054504
0.39322322607040405
0.2182573676109314
0.46268749237060547
0.2525574862957001
0.8972026705741882
0.5239914059638977
0.47450631856918335
0.2114875614643097
0.24918143451213837
0.6901196241378784
0.2042209357023239
0.1671942174434662
0.3018033802509308
0.3821534514427185
0.30768513679504395
0.20434530079364777
0.8972026705741882
0.4575531780719757
0.4463531970977783
0.4463531970977783
0.4463531970977783
0.4797777533531189
0.3748798668384552
0.28588879108428955
0.24968449771404266
0.3022747039794922
0.37681812047958374
0.49019306898117065
0.5532710552215576
0.2873285710811615
0.41965094208717346
0.4463531970977783
0.4463531970977783
0.13503587245941162
0.7923744320869446
0.23042380809783936
0.36761194467544556
0.37133222818374634
0.351715087890625
0.5242227911949158
0.3841732144355774
0.38417321443

0.11408208310604095
0.17016199231147766
0.3551930785179138
0.3551930785179138
0.3551930785179138
0.19055567681789398
0.287946879863739
0.11237876862287521
0.10789443552494049
0.05031251907348633
0.2126310169696808
0.43819430470466614
0.4852466583251953
0.3712914288043976
0.49524545669555664
0.3551930785179138
0.3551930785179138
0.41569530963897705
0.25693556666374207
0.1683913767337799
0.5155019164085388
0.4378627836704254
0.25142011046409607
0.37191030383110046
0.42938509583473206
0.42938509583473206
0.308669775724411
0.049286406487226486
0.4170213043689728
0.1985890120267868
0.4374772310256958
0.3968614637851715
0.38454046845436096
0.5149292945861816
0.35422638058662415
0.5295965671539307
0.23763880133628845
0.3936721682548523
0.22894468903541565
0.48307037353515625
0.16022354364395142
0.3731556236743927
0.32236558198928833
0.33426663279533386
0.531036913394928
0.2885953187942505
0.2885953187942505
0.2237854152917862
0.4314541518688202
0.5005245208740234
0.8972026705741882
0.24320748

0.123469777405262
0.23867808282375336
0.42712461948394775
0.3319089710712433
0.28707191348075867
0.039370257407426834
-0.021718358621001244
0.6005571484565735
0.20801319181919098
0.5472766757011414
0.21309390664100647
0.2003239393234253
0.404928058385849
0.15466558933258057
0.5523784160614014
0.2975536286830902
0.48061296343803406
0.16464903950691223
0.22510796785354614
0.4034377932548523
0.35165220499038696
0.3480331003665924
0.11027291417121887
0.22488316893577576
0.5541114211082458
0.5016397833824158
0.48197999596595764
0.3057648837566376
0.1899140626192093
0.2857384979724884
0.4170213043689728
0.1985890120267868
0.4374772310256958
0.3968614637851715
0.38454046845436096
0.5149292945861816
0.35422638058662415
0.5295965671539307
0.23763880133628845
0.3936721682548523
0.22894468903541565
0.48307037353515625
0.16022354364395142
0.3731556236743927
0.32236558198928833
0.33426663279533386
0.531036913394928
0.2885953187942505
0.2885953187942505
0.2237854152917862
0.4314541518688202
0.572644

0.24060209095478058
0.20314064621925354
0.18300607800483704
0.20230180025100708
0.30211764574050903
0.3234666883945465
0.22788859903812408
0.3610030710697174
0.1904979795217514
0.5005245208740234
0.13215434551239014
0.22101929783821106
0.17016199231147766
0.2872270941734314
0.3018586337566376
0.192576602101326
0.30768513679504395
0.15890224277973175
0.29247763752937317
0.2108749896287918
0.38959887623786926
0.23722979426383972
0.177383691072464
0.2625240981578827
0.031928833574056625
0.3267054855823517
0.14362949132919312
0.14362949132919312
0.3170754611492157
0.2849518060684204
0.10734561830759048
0.1683979481458664
0.15014593303203583
0.42865538597106934
0.11086501181125641
0.20644348859786987
0.05601459741592407
0.16827282309532166
0.08771379292011261
0.2467503398656845
0.22004321217536926
0.16718482971191406
0.07212583720684052
0.30570802092552185
0.4143579304218292
0.3630084991455078
0.2107013612985611
0.21177449822425842
0.38260695338249207
0.6195286512374878
0.33880212903022766


0.08058293908834457
-0.03404329717159271
0.009173517115414143
0.06628256291151047
0.08367835730314255
0.5239914059638977
0.2132752537727356
0.7234411835670471
0.34797579050064087
0.2237854152917862
0.2237854152917862
0.222797229886055
0.19337345659732819
0.05051105469465256
0.2833509147167206
0.2427520453929901
0.17016199231147766
0.30726590752601624
0.5005245208740234
0.5005245208740234
0.01717686466872692


In [51]:
sent_kp_df

,arg_id,topic,argument,num_of_aspects,domain,aspects_x,opinions_x,opinion_aspect_pairs_x,sentiments_x,attributes_x,key_point_id,key_point,aspects_y,opinions_y,opinion_aspect_pairs_y,sentiments_y,attributes_y,predicted_WA,score,predicted_by_cosine
0,arg_0_537_2,Arts & Entertainment,Employees are rude and make the experience uninviting .,2,Arts & Entertainment,experience,uninviting,uninviting experience,negative,restaurant -> atmosphere,kp_0_2,Terrible administration and management .,administration,terrible,terrible administration,negative,staff,0.434350193,0.479240,True
1,arg_0_537_2,Arts & Entertainment,Employees are rude and make the experience uninviting .,2,Arts & Entertainment,experience,uninviting,uninviting experience,negative,restaurant -> atmosphere,kp_0_24,Rude and Unprofessional staff .,staff,unprofessional,unprofessional staff,negative,staff,0.618978381,0.404928,True
2,arg_0_537_2,Arts & Entertainment,Employees are rude and make the experience uninviting .,2,Arts & Entertainment,experience,uninviting,uninviting experience,negative,restaurant -> atmosphere,kp_0_4,Great for conventions and concerts ! !,conventions,great for,great for conventions,positive,food -> variety,0.457361251,0.000000,False
3,arg_0_537_2,Arts & Entertainment,Employees are rude and make the experience uninviting .,2,Arts & Entertainment,experience,uninviting,uninviting experience,negative,restaurant -> atmosphere,kp_0_5,Horrible customer service .,customer service,horrible,horrible customer service,negative,staff,0.477881193,0.604674,True
4,arg_0_537_2,Arts & Entertainment,Employees are rude and make the experience uninviting .,2,Arts & Entertainment,experience,uninviting,uninviting experience,negative,restaurant -> atmosphere,kp_0_0,It had wonderful acoustics .,acoustics,wonderful,wonderful acoustics,positive,restaurant -> atmosphere,0.574064255,0.000000,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8715,arg_4_6440_1,Restaurants,The staff are friendly and attentive .,1,Restaurants,staff,friendly,friendly staff,positive,staff,kp_4_28,The service here was exceptional .,service,exceptional,exceptional service,positive,staff,0.454417199,0.500525,True
8716,arg_4_6440_1,Restaurants,The staff are friendly and attentive .,1,Restaurants,staff,friendly,friendly staff,positive,staff,kp_4_29,Staff was courteous and accommodating .,staff,courteous,courteous staff,positive,staff,0.671794057,1.000000,False
8717,arg_4_6440_1,Restaurants,The staff are friendly and attentive .,1,Restaurants,staff,friendly,friendly staff,positive,staff,kp_4_30,Rude and inattentive staff .,staff,inattentive,inattentive staff,negative,staff,0.548471034,0.000000,False
8718,arg_4_6440_1,Restaurants,The staff are friendly and attentive .,1,Restaurants,staff,friendly,friendly staff,positive,staff,kp_4_31,Great sushi and sashimi .,sushi,great,great sushi,positive,food -> quality,0.43206653,0.017177,True


In [52]:
import nltk
def label_implicit_matching_pair(row):
    token1 = row['aspects_x'][0]
    token2 = row['aspects_y'][0]
    
    # Best
    if row['aspects_x_len'] >= row['aspects_y_len']:
        if token2 in token1 and len(token1) > len(token2) and row['aspects_x_len'] > row['aspects_y_len']: # e.g. mexican food vs food
            row['label'] = 1
    return row

In [53]:
# matched_sent_kp_df[matched_sent_kp_df['aspects_similarity'] > 0.55] = matched_sent_kp_df[matched_sent_kp_df['aspects_similarity'] > 0.55].apply(label_implicit_matching_pair, axis=1)

In [54]:
# mask = (sent_kp_df['predicted_by_cosine'] == True)
# # sent_kp_df.loc[mask & (sent_kp_df['score'] > 0.55), 'score'] = 1
# sent_kp_df.loc[mask & (sent_kp_df['score'] <= 0.55), 'score'] = 0

In [55]:
sent_kp_df

,arg_id,topic,argument,num_of_aspects,domain,aspects_x,opinions_x,opinion_aspect_pairs_x,sentiments_x,attributes_x,key_point_id,key_point,aspects_y,opinions_y,opinion_aspect_pairs_y,sentiments_y,attributes_y,predicted_WA,score,predicted_by_cosine
0,arg_0_537_2,Arts & Entertainment,Employees are rude and make the experience uninviting .,2,Arts & Entertainment,experience,uninviting,uninviting experience,negative,restaurant -> atmosphere,kp_0_2,Terrible administration and management .,administration,terrible,terrible administration,negative,staff,0.434350193,0.479240,True
1,arg_0_537_2,Arts & Entertainment,Employees are rude and make the experience uninviting .,2,Arts & Entertainment,experience,uninviting,uninviting experience,negative,restaurant -> atmosphere,kp_0_24,Rude and Unprofessional staff .,staff,unprofessional,unprofessional staff,negative,staff,0.618978381,0.404928,True
2,arg_0_537_2,Arts & Entertainment,Employees are rude and make the experience uninviting .,2,Arts & Entertainment,experience,uninviting,uninviting experience,negative,restaurant -> atmosphere,kp_0_4,Great for conventions and concerts ! !,conventions,great for,great for conventions,positive,food -> variety,0.457361251,0.000000,False
3,arg_0_537_2,Arts & Entertainment,Employees are rude and make the experience uninviting .,2,Arts & Entertainment,experience,uninviting,uninviting experience,negative,restaurant -> atmosphere,kp_0_5,Horrible customer service .,customer service,horrible,horrible customer service,negative,staff,0.477881193,0.604674,True
4,arg_0_537_2,Arts & Entertainment,Employees are rude and make the experience uninviting .,2,Arts & Entertainment,experience,uninviting,uninviting experience,negative,restaurant -> atmosphere,kp_0_0,It had wonderful acoustics .,acoustics,wonderful,wonderful acoustics,positive,restaurant -> atmosphere,0.574064255,0.000000,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8715,arg_4_6440_1,Restaurants,The staff are friendly and attentive .,1,Restaurants,staff,friendly,friendly staff,positive,staff,kp_4_28,The service here was exceptional .,service,exceptional,exceptional service,positive,staff,0.454417199,0.500525,True
8716,arg_4_6440_1,Restaurants,The staff are friendly and attentive .,1,Restaurants,staff,friendly,friendly staff,positive,staff,kp_4_29,Staff was courteous and accommodating .,staff,courteous,courteous staff,positive,staff,0.671794057,1.000000,False
8717,arg_4_6440_1,Restaurants,The staff are friendly and attentive .,1,Restaurants,staff,friendly,friendly staff,positive,staff,kp_4_30,Rude and inattentive staff .,staff,inattentive,inattentive staff,negative,staff,0.548471034,0.000000,False
8718,arg_4_6440_1,Restaurants,The staff are friendly and attentive .,1,Restaurants,staff,friendly,friendly staff,positive,staff,kp_4_31,Great sushi and sashimi .,sushi,great,great sushi,positive,food -> quality,0.43206653,0.017177,True


In [56]:
# import nltk
# def label_implicit_matching_pair(row):
#     token1 = row['aspects_x'][0]
#     token2 = row['aspects_y'][0]
    
#     # Best
#     if row['aspects_x_len'] >= row['aspects_y_len']:
#         if token2 in token1 and len(token1) > len(token2) and row['aspects_x_len'] > row['aspects_y_len']: # e.g. mexican food vs food
#             row['label'] = 1

# #     if row['aspects_x_len'] >=     w['aspects_y_len']:
# #         if (token2 in token1) and len(token1) > len(token2): # e.g. mexican food vs food, prices vs price
# #             row['label'] = 1
# #             if row['aspects_x_len'] > row['aspects_y_len']:
# #                 display(row)

# #     elif row['aspects_x_len'] == row['aspects_y_len']:
# #         if nltk.edit_distance(token1, token2) <= 2 and len(token1) >= len(token2): # e.g. prices, price
# #             display(row)
# #             row['label'] = 1


# #     if row['aspects_x_len'] >= row['aspects_y_len']:
# #         if (token2 in token1 or nltk.edit_distance(token1, token2) <= 2) and\
# #             len(token1) >= len(token2): # e.g. mexican food vs food
# #             row['label'] = 1


            
# #     if token2 in token1 and len(token1) > len(token2) and row['aspects_x_len'] > row['aspects_y_len']: # e.g. mexican food vs food
# #         row['label'] = 2
# #     elif row['aspects_x_len'] == row['aspects_y_len']:
# #         row['label'] = 2
    
#     return row

In [57]:
# matched_sent_kp_df[matched_sent_kp_df['aspects_similarity'] > 0.55] = matched_sent_kp_df[matched_sent_kp_df['aspects_similarity'] > 0.55].apply(label_implicit_matching_pair, axis=1)

In [58]:
sent_kp_df

,arg_id,topic,argument,num_of_aspects,domain,aspects_x,opinions_x,opinion_aspect_pairs_x,sentiments_x,attributes_x,key_point_id,key_point,aspects_y,opinions_y,opinion_aspect_pairs_y,sentiments_y,attributes_y,predicted_WA,score,predicted_by_cosine
0,arg_0_537_2,Arts & Entertainment,Employees are rude and make the experience uninviting .,2,Arts & Entertainment,experience,uninviting,uninviting experience,negative,restaurant -> atmosphere,kp_0_2,Terrible administration and management .,administration,terrible,terrible administration,negative,staff,0.434350193,0.479240,True
1,arg_0_537_2,Arts & Entertainment,Employees are rude and make the experience uninviting .,2,Arts & Entertainment,experience,uninviting,uninviting experience,negative,restaurant -> atmosphere,kp_0_24,Rude and Unprofessional staff .,staff,unprofessional,unprofessional staff,negative,staff,0.618978381,0.404928,True
2,arg_0_537_2,Arts & Entertainment,Employees are rude and make the experience uninviting .,2,Arts & Entertainment,experience,uninviting,uninviting experience,negative,restaurant -> atmosphere,kp_0_4,Great for conventions and concerts ! !,conventions,great for,great for conventions,positive,food -> variety,0.457361251,0.000000,False
3,arg_0_537_2,Arts & Entertainment,Employees are rude and make the experience uninviting .,2,Arts & Entertainment,experience,uninviting,uninviting experience,negative,restaurant -> atmosphere,kp_0_5,Horrible customer service .,customer service,horrible,horrible customer service,negative,staff,0.477881193,0.604674,True
4,arg_0_537_2,Arts & Entertainment,Employees are rude and make the experience uninviting .,2,Arts & Entertainment,experience,uninviting,uninviting experience,negative,restaurant -> atmosphere,kp_0_0,It had wonderful acoustics .,acoustics,wonderful,wonderful acoustics,positive,restaurant -> atmosphere,0.574064255,0.000000,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8715,arg_4_6440_1,Restaurants,The staff are friendly and attentive .,1,Restaurants,staff,friendly,friendly staff,positive,staff,kp_4_28,The service here was exceptional .,service,exceptional,exceptional service,positive,staff,0.454417199,0.500525,True
8716,arg_4_6440_1,Restaurants,The staff are friendly and attentive .,1,Restaurants,staff,friendly,friendly staff,positive,staff,kp_4_29,Staff was courteous and accommodating .,staff,courteous,courteous staff,positive,staff,0.671794057,1.000000,False
8717,arg_4_6440_1,Restaurants,The staff are friendly and attentive .,1,Restaurants,staff,friendly,friendly staff,positive,staff,kp_4_30,Rude and inattentive staff .,staff,inattentive,inattentive staff,negative,staff,0.548471034,0.000000,False
8718,arg_4_6440_1,Restaurants,The staff are friendly and attentive .,1,Restaurants,staff,friendly,friendly staff,positive,staff,kp_4_31,Great sushi and sashimi .,sushi,great,great sushi,positive,food -> quality,0.43206653,0.017177,True


In [59]:
def select_best_kp_per_opinion(grp):
    best_match = grp.sort_values(by=['score'], ascending=False)
    best_match = best_match.iloc[0]
    return best_match
    
new_sent_kp_df = sent_kp_df.groupby(['topic', 'arg_id', 'opinion_aspect_pairs_x']).apply(select_best_kp_per_opinion).reset_index(drop=True)

In [60]:
new_sent_kp_df = new_sent_kp_df.drop_duplicates(subset=['arg_id', 'key_point_id'])
new_sent_kp_df

,arg_id,topic,argument,num_of_aspects,domain,aspects_x,opinions_x,opinion_aspect_pairs_x,sentiments_x,attributes_x,key_point_id,key_point,aspects_y,opinions_y,opinion_aspect_pairs_y,sentiments_y,attributes_y,predicted_WA,score,predicted_by_cosine
0,arg_0_1186_1,Arts & Entertainment,I love her musical talent and the range of her voice .,1,Arts & Entertainment,musical talent,love,love musical talent,positive,restaurant -> atmosphere,kp_0_16,Very talented musicians !,musicians,very talented,very talented musicians,positive,restaurant -> atmosphere,0.432230115,0.705463,True
1,arg_0_1227_1,Arts & Entertainment,I loved the design- lots of green and water features .,1,Arts & Entertainment,design,loved,loved design,positive,restaurant -> atmosphere,kp_0_13,"Awesome lighting , and decor .",lighting,awesome,awesome lighting,positive,restaurant -> atmosphere,0.43309477,0.627256,True
2,arg_0_1459_2,Arts & Entertainment,It also has great amenities and the seats are comfortable .,2,Arts & Entertainment,seats,comfortable,comfortable seats,positive,restaurant -> comfort,kp_0_22,Seats are adequately comfortable .,seats,adequately comfortable,adequately comfortable seats,positive,restaurant -> comfort,0.442808837,1.000000,False
3,arg_0_1459_2,Arts & Entertainment,It also has great amenities and the seats are comfortable .,2,Arts & Entertainment,amenities,great,great amenities,positive,restaurant -> atmosphere,kp_0_17,Parking was fast and FREE !,parking,fast and free,fast and free parking,positive,wait-time,0.572399199,0.576591,True
4,arg_0_1594_1,Arts & Entertainment,Lobby concessions have improved in both quality and wait time .,1,Arts & Entertainment,lobby concessions,improved,improved lobby concessions,negative,restaurant -> atmosphere,kp_0_2,Terrible administration and management .,administration,terrible,terrible administration,negative,staff,0.434350193,0.606139,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
298,arg_4_768_2,Restaurants,Casual place for brunch with consistently good food and service,2,Restaurants,place for,casual,casual place for,positive,recommendation,kp_4_19,Lots of outdoor seating and,outdoor seating,lots of,lots of outdoor seating,positive,restaurant -> atmosphere,0.466573238,0.515502,True
299,arg_4_768_2,Restaurants,Casual place for brunch with consistently good food and service,2,Restaurants,food,consistently good,consistently good food,positive,food -> quality,kp_4_11,"Fresh food , using local produce .",food,fresh,fresh food,positive,food -> quality,0.621725261,1.000000,False
300,arg_4_7703_1,Restaurants,We sat at the bar and received excellent service from Matthew .,1,Restaurants,service,excellent,excellent service,positive,staff,kp_4_28,The service here was exceptional .,service,exceptional,exceptional service,positive,staff,0.454417199,1.000000,False
301,arg_4_81_1,Restaurants,A great place for a casual breakfast or lunch .,1,Restaurants,place for a casual breakfast or lunch,great,great place for a casual breakfast or lunch,positive,food -> quality,kp_4_7,Fantastic drink selection .,drink selection,fantastic,fantastic drink selection,positive,drink -> quality,0.475231081,0.553658,True


In [61]:
merged_df = pd.merge(new_sent_kp_df, labels_df, how="left", on=["arg_id", "key_point_id"])

In [62]:
# #merge each argument with the gold labels
# merged_df = pd.merge(sent_kp_df, labels_df, how="left", on=["arg_id", "key_point_id"])

In [63]:
merged_df.loc[merged_df['key_point_id'] == "dummy_id", 'label'] = 0
merged_df["label_strict"] = merged_df["label"].fillna(0)
merged_df["label_relaxed"] = merged_df["label"].fillna(1)

In [64]:
merged_df

,arg_id,topic,argument,num_of_aspects,domain,aspects_x,opinions_x,opinion_aspect_pairs_x,sentiments_x,attributes_x,...,opinions_y,opinion_aspect_pairs_y,sentiments_y,attributes_y,predicted_WA,score,predicted_by_cosine,label,label_strict,label_relaxed
0,arg_0_1186_1,Arts & Entertainment,I love her musical talent and the range of her voice .,1,Arts & Entertainment,musical talent,love,love musical talent,positive,restaurant -> atmosphere,...,very talented,very talented musicians,positive,restaurant -> atmosphere,0.432230115,0.705463,True,1,1,1
1,arg_0_1227_1,Arts & Entertainment,I loved the design- lots of green and water features .,1,Arts & Entertainment,design,loved,loved design,positive,restaurant -> atmosphere,...,awesome,awesome lighting,positive,restaurant -> atmosphere,0.43309477,0.627256,True,1,1,1
2,arg_0_1459_2,Arts & Entertainment,It also has great amenities and the seats are comfortable .,2,Arts & Entertainment,seats,comfortable,comfortable seats,positive,restaurant -> comfort,...,adequately comfortable,adequately comfortable seats,positive,restaurant -> comfort,0.442808837,1.000000,False,1,1,1
3,arg_0_1459_2,Arts & Entertainment,It also has great amenities and the seats are comfortable .,2,Arts & Entertainment,amenities,great,great amenities,positive,restaurant -> atmosphere,...,fast and free,fast and free parking,positive,wait-time,0.572399199,0.576591,True,0,0,0
4,arg_0_1594_1,Arts & Entertainment,Lobby concessions have improved in both quality and wait time .,1,Arts & Entertainment,lobby concessions,improved,improved lobby concessions,negative,restaurant -> atmosphere,...,terrible,terrible administration,negative,staff,0.434350193,0.606139,True,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
286,arg_4_768_2,Restaurants,Casual place for brunch with consistently good food and service,2,Restaurants,place for,casual,casual place for,positive,recommendation,...,lots of,lots of outdoor seating,positive,restaurant -> atmosphere,0.466573238,0.515502,True,0,0,0
287,arg_4_768_2,Restaurants,Casual place for brunch with consistently good food and service,2,Restaurants,food,consistently good,consistently good food,positive,food -> quality,...,fresh,fresh food,positive,food -> quality,0.621725261,1.000000,False,0,0,0
288,arg_4_7703_1,Restaurants,We sat at the bar and received excellent service from Matthew .,1,Restaurants,service,excellent,excellent service,positive,staff,...,exceptional,exceptional service,positive,staff,0.454417199,1.000000,False,0,0,0
289,arg_4_81_1,Restaurants,A great place for a casual breakfast or lunch .,1,Restaurants,place for a casual breakfast or lunch,great,great place for a casual breakfast or lunch,positive,food -> quality,...,fantastic,fantastic drink selection,positive,drink -> quality,0.475231081,0.553658,True,0,0,0


In [65]:
def get_ap(df, label_column, top_percentile=0.5):
    top = int(len(df)*top_percentile)
    df = df.sort_values('score', ascending=False)
    if top != 0:
        df = df.head(top)
    # after selecting top percentile candidates, we set the score for the dummy kp to 1, to prevent it from increasing the precision.
    df.loc[df['key_point_id'] == "dummy_id", 'score'] = 0.99
#     print("NEW")
#     display(df)
#     display(df[pd.isnull(df)])
#     print()
    return average_precision_score(y_true=df[label_column], y_score=df["score"])

def calc_mean_average_precision(df, label_column):
    precisions = [get_ap(group, label_column) for _, group in df.groupby(["topic"])]
#     precisions = [0 if pd.isnull(pre) else pre for pre in precisions]
#     precisions = [pre for pre in precisions if pd.notnull(pre)]
    print(precisions)
    return np.mean(precisions)

def evaluate_predictions(merged_df):
    mAP_strict = calc_mean_average_precision(merged_df, "label_strict")
    mAP_relaxed = calc_mean_average_precision(merged_df, "label_relaxed")
    return mAP_strict, mAP_relaxed

In [66]:
[(t, get_ap(group, "label_strict")) for t, group in merged_df.groupby(["topic"])]

[('Arts & Entertainment', 0.92),
 ('Automotive', 0.5822776003554035),
 ('Beauty & Spas', 0.8496957587866678),
 ('Hotels', 0.9538051289328429),
 ('Restaurants', 0.7765644827355906)]

In [67]:
[(t, get_ap(group, "label_relaxed")) for t, group in merged_df.groupby(["topic"])]

[('Arts & Entertainment', 0.92),
 ('Automotive', 0.5822776003554035),
 ('Beauty & Spas', 0.8496957587866678),
 ('Hotels', 0.9538051289328429),
 ('Restaurants', 0.7765644827355906)]

In [63]:
# Topic merge only, with best-matching
# 30%
#Perform evaluation
mAP_strict, mAP_relaxed = evaluate_predictions(merged_df)

print(f"mAP strict= {mAP_strict} ; mAP relaxed = {mAP_relaxed}")

# logger.info("mAP strict:   \t{:.2f}".format(mAP_strict*100))
# logger.info("mAP relaxed:   \t{:.2f}".format(mAP_relaxed*100))

[0.8962692164780265, 0.8068616317097077]
[0.8962692164780265, 0.8068616317097077]
mAP strict= 0.851565424093867 ; mAP relaxed = 0.851565424093867


### Hotels

Multi-Aspect

In [64]:
# Topic merge only, with best-matching
# 30%
#Perform evaluation
mAP_strict, mAP_relaxed = evaluate_predictions(merged_df)

print(f"mAP strict= {mAP_strict} ; mAP relaxed = {mAP_relaxed}")

# logger.info("mAP strict:   \t{:.2f}".format(mAP_strict*100))
# logger.info("mAP relaxed:   \t{:.2f}".format(mAP_relaxed*100))

[0.879320987654321]
[0.879320987654321]
mAP strict= 0.879320987654321 ; mAP relaxed = 0.879320987654321


Combined

In [65]:
# Topic merge only, with best-matching
# 30%
#Perform evaluation
mAP_strict, mAP_relaxed = evaluate_predictions(merged_df)

print(f"mAP strict= {mAP_strict} ; mAP relaxed = {mAP_relaxed}")

# logger.info("mAP strict:   \t{:.2f}".format(mAP_strict*100))
# logger.info("mAP relaxed:   \t{:.2f}".format(mAP_relaxed*100))

[0.9538051289328429]
[0.9538051289328429]
mAP strict= 0.9538051289328429 ; mAP relaxed = 0.9538051289328429


### Restaurants

Multi-Aspect

In [69]:
# Topic merge only, with best-matching
# 30%
#Perform evaluation
mAP_strict, mAP_relaxed = evaluate_predictions(merged_df)

print(f"mAP strict= {mAP_strict} ; mAP relaxed = {mAP_relaxed}")

# logger.info("mAP strict:   \t{:.2f}".format(mAP_strict*100))
# logger.info("mAP relaxed:   \t{:.2f}".format(mAP_relaxed*100))

[0.716017316017316]
[0.716017316017316]
mAP strict= 0.716017316017316 ; mAP relaxed = 0.716017316017316


Combined

In [63]:
# Topic merge only, with best-matching
# 30%
#Perform evaluation
mAP_strict, mAP_relaxed = evaluate_predictions(merged_df)

print(f"mAP strict= {mAP_strict} ; mAP relaxed = {mAP_relaxed}")

# logger.info("mAP strict:   \t{:.2f}".format(mAP_strict*100))
# logger.info("mAP relaxed:   \t{:.2f}".format(mAP_relaxed*100))

[0.7765644827355906]
[0.7765644827355906]
mAP strict= 0.7765644827355906 ; mAP relaxed = 0.7765644827355906


### Arts & Entertainment

Multi-Aspect

In [66]:
# Topic merge only, with best-matching
# 30%
#Perform evaluation
mAP_strict, mAP_relaxed = evaluate_predictions(merged_df)

print(f"mAP strict= {mAP_strict} ; mAP relaxed = {mAP_relaxed}")

# logger.info("mAP strict:   \t{:.2f}".format(mAP_strict*100))
# logger.info("mAP relaxed:   \t{:.2f}".format(mAP_relaxed*100))

[0.92]
[0.92]
mAP strict= 0.92 ; mAP relaxed = 0.92


Combined

In [66]:
# Topic merge only, with best-matching
# 30%
#Perform evaluation
mAP_strict, mAP_relaxed = evaluate_predictions(merged_df)

print(f"mAP strict= {mAP_strict} ; mAP relaxed = {mAP_relaxed}")

# logger.info("mAP strict:   \t{:.2f}".format(mAP_strict*100))
# logger.info("mAP relaxed:   \t{:.2f}".format(mAP_relaxed*100))

[0.9230769230769231]
[0.9230769230769231]
mAP strict= 0.9230769230769231 ; mAP relaxed = 0.9230769230769231


### Automotive

Multi-Aspect

In [69]:
# Topic merge only, with best-matching
# 30%
#Perform evaluation
mAP_strict, mAP_relaxed = evaluate_predictions(merged_df)

print(f"mAP strict= {mAP_strict} ; mAP relaxed = {mAP_relaxed}")

# logger.info("mAP strict:   \t{:.2f}".format(mAP_strict*100))
# logger.info("mAP relaxed:   \t{:.2f}".format(mAP_relaxed*100))

[0.42857142857142855]
[0.42857142857142855]
mAP strict= 0.42857142857142855 ; mAP relaxed = 0.42857142857142855


Combined

In [63]:
# Topic merge only, with best-matching
# 30%
#Perform evaluation
mAP_strict, mAP_relaxed = evaluate_predictions(merged_df)

print(f"mAP strict= {mAP_strict} ; mAP relaxed = {mAP_relaxed}")

# logger.info("mAP strict:   \t{:.2f}".format(mAP_strict*100))
# logger.info("mAP relaxed:   \t{:.2f}".format(mAP_relaxed*100))

[0.5822776003554035]
[0.5822776003554035]
mAP strict= 0.5822776003554035 ; mAP relaxed = 0.5822776003554035


### Beauty & Spas

Multi-Aspect

In [69]:
# Topic merge only, with best-matching
# 30%
#Perform evaluation
mAP_strict, mAP_relaxed = evaluate_predictions(merged_df)

print(f"mAP strict= {mAP_strict} ; mAP relaxed = {mAP_relaxed}")

# logger.info("mAP strict:   \t{:.2f}".format(mAP_strict*100))
# logger.info("mAP relaxed:   \t{:.2f}".format(mAP_relaxed*100))

[0.823639455782313]
[0.823639455782313]
mAP strict= 0.823639455782313 ; mAP relaxed = 0.823639455782313


Combined

In [63]:
# Topic merge only, with best-matching
# 30%
#Perform evaluation
mAP_strict, mAP_relaxed = evaluate_predictions(merged_df)

print(f"mAP strict= {mAP_strict} ; mAP relaxed = {mAP_relaxed}")

# logger.info("mAP strict:   \t{:.2f}".format(mAP_strict*100))
# logger.info("mAP relaxed:   \t{:.2f}".format(mAP_relaxed*100))

[0.8496957587866678]
[0.8496957587866678]
mAP strict= 0.8496957587866678 ; mAP relaxed = 0.8496957587866678


Multi-aspect

In [55]:
# Topic merge only, with best-matching
# 30%
#Perform evaluation
mAP_strict, mAP_relaxed = evaluate_predictions(merged_df)

print(f"mAP strict= {mAP_strict} ; mAP relaxed = {mAP_relaxed}")

# logger.info("mAP strict:   \t{:.2f}".format(mAP_strict*100))
# logger.info("mAP relaxed:   \t{:.2f}".format(mAP_relaxed*100))

[0.8844444444444444, 0.7924607706370156]
[0.8844444444444444, 0.7924607706370156]
mAP strict= 0.8384526075407299 ; mAP relaxed = 0.8384526075407299


In [ ]:
# Topic merge only, with no best-matching
# Only 0.42

In [317]:
# Topic merge only, with best-matching
# 30%
#Perform evaluation
mAP_strict, mAP_relaxed = evaluate_predictions(merged_df)

print(f"mAP strict= {mAP_strict} ; mAP relaxed = {mAP_relaxed}")

# logger.info("mAP strict:   \t{:.2f}".format(mAP_strict*100))
# logger.info("mAP relaxed:   \t{:.2f}".format(mAP_relaxed*100))

[0.8962692164780265, 0.8068616317097077]
[0.8962692164780265, 0.8068616317097077]
mAP strict= 0.851565424093867 ; mAP relaxed = 0.851565424093867


In [247]:
# Topic vs aspect category merge, but no best-matching
# 30%
#Perform evaluation
mAP_strict, mAP_relaxed = evaluate_predictions(merged_df)

print(f"mAP strict= {mAP_strict} ; mAP relaxed = {mAP_relaxed}")

# logger.info("mAP strict:   \t{:.2f}".format(mAP_strict*100))
# logger.info("mAP relaxed:   \t{:.2f}".format(mAP_relaxed*100))

[0.7078265760989924, 0.6041469582888587]
[0.7078265760989924, 0.6041469582888587]
mAP strict= 0.6559867671939255 ; mAP relaxed = 0.6559867671939255


In [262]:
# 30%
# Topic vs aspect category merge, with best-matching
#Perform evaluation
mAP_strict, mAP_relaxed = evaluate_predictions(merged_df)

print(f"mAP strict= {mAP_strict} ; mAP relaxed = {mAP_relaxed}")

# logger.info("mAP strict:   \t{:.2f}".format(mAP_strict*100))
# logger.info("mAP relaxed:   \t{:.2f}".format(mAP_relaxed*100))

[0.9216653367978361, 0.8017873848903975]
[0.9216653367978361, 0.8017873848903975]
mAP strict= 0.8617263608441168 ; mAP relaxed = 0.8617263608441168


With 0.55 Rounding Neg Only

In [366]:
# 30%
#Perform evaluation
mAP_strict, mAP_relaxed = evaluate_predictions(merged_df)

print(f"mAP strict= {mAP_strict} ; mAP relaxed = {mAP_relaxed}")

# logger.info("mAP strict:   \t{:.2f}".format(mAP_strict*100))
# logger.info("mAP relaxed:   \t{:.2f}".format(mAP_relaxed*100))

[0.5524090137049532, 0.40549477577266085]
[0.5524090137049532, 0.40549477577266085]
mAP strict= 0.47895189473880706 ; mAP relaxed = 0.47895189473880706


In [381]:
# 30%
#Perform evaluation
mAP_strict, mAP_relaxed = evaluate_predictions(merged_df)

print(f"mAP strict= {mAP_strict} ; mAP relaxed = {mAP_relaxed}")

# logger.info("mAP strict:   \t{:.2f}".format(mAP_strict*100))
# logger.info("mAP relaxed:   \t{:.2f}".format(mAP_relaxed*100))

[0.9314898262807844, 0.8012792037619579]
[0.9314898262807844, 0.8012792037619579]
mAP strict= 0.8663845150213711 ; mAP relaxed = 0.8663845150213711


With 0.55 Rounding Pos, Neg

In [390]:
# 30%
#Perform evaluation
mAP_strict, mAP_relaxed = evaluate_predictions(merged_df)

print(f"mAP strict= {mAP_strict} ; mAP relaxed = {mAP_relaxed}")

# logger.info("mAP strict:   \t{:.2f}".format(mAP_strict*100))
# logger.info("mAP relaxed:   \t{:.2f}".format(mAP_relaxed*100))

[0.2771587151629665, 0.19557911566790592]
[0.2771587151629665, 0.19557911566790592]
mAP strict= 0.2363689154154362 ; mAP relaxed = 0.2363689154154362


In [396]:
# 30%
#Perform evaluation
mAP_strict, mAP_relaxed = evaluate_predictions(merged_df)

print(f"mAP strict= {mAP_strict} ; mAP relaxed = {mAP_relaxed}")

# logger.info("mAP strict:   \t{:.2f}".format(mAP_strict*100))
# logger.info("mAP relaxed:   \t{:.2f}".format(mAP_relaxed*100))

[0.41304347826086957, 0.3333333333333333]
[0.41304347826086957, 0.3333333333333333]
mAP strict= 0.37318840579710144 ; mAP relaxed = 0.37318840579710144


In [322]:
# 30%
#Perform evaluation
mAP_strict, mAP_relaxed = evaluate_predictions(merged_df)

print(f"mAP strict= {mAP_strict} ; mAP relaxed = {mAP_relaxed}")

# logger.info("mAP strict:   \t{:.2f}".format(mAP_strict*100))
# logger.info("mAP relaxed:   \t{:.2f}".format(mAP_relaxed*100))

[0.4690003763949305, 0.3862436420715855]
[0.4690003763949305, 0.3862436420715855]
mAP strict= 0.427622009233258 ; mAP relaxed = 0.427622009233258


### Aspect-based Matching KP Coverage Test 2

In [73]:
valid_df = df[(df['label'] == 1) & (df['arg_id'].str.startswith('arg_3'))]
valid_kp = valid_df['full_key_point'].value_counts()
valid_kp

Friendly and helpful staff .              21
Clean and comfortable rooms .             13
Excellent customer service .               7
The service is impeccable .                5
Very peaceful and relaxing location .      4
Great desk service , too .                 4
Extremely poor service .                   3
The beds are extremely comfortable .       2
Pool area clean and refreshing .           2
Price way overvalued .                     1
The ambience is wonderfully peaceful .     1
Horribly rude staff .                      1
Front desk staff was inept .               1
Room decor is outdated .                   1
Wifi was very reliable .                   1
Valet is convenient and fast .             1
Parking is easy .                          1
Restaurant service absolutely sucks .      1
Beds were extremely uncomfortable .        1
Great cocktail list too .                  1
Name: full_key_point, dtype: int64

In [74]:
# valid_kp[valid_kp > 3]

In [75]:
# merged_df[merged_df['arg_id'].str.startswith('arg_3')].merge(kp_df, on=['key_point_id'])['full_key_point'].tolist()

In [76]:
merged_df.columns

Index(['arg_id', 'topic', 'argument', 'num_of_aspects', 'aspects_x',
       'opinions_x', 'opinion_aspect_pairs_x', 'sentiments_x', 'attributes_x',
       'key_point_id', 'key_point', 'aspects_y', 'opinions_y',
       'opinion_aspect_pairs_y', 'sentiments_y', 'attributes_y',
       'predicted_WA', 'score', 'predicted_by_cosine', 'label', 'label_strict',
       'label_relaxed'],
      dtype='object')

In [77]:
# Hotel Only
top_kp = merged_df[merged_df['arg_id'].str.startswith('arg_3')].merge(kp_df, on=['key_point_id'])['key_point_x'].value_counts()
result = top_kp[top_kp.index.isin(valid_kp.index.tolist())]
result

Friendly and helpful staff .              16
Clean and comfortable rooms .             12
Very peaceful and relaxing location .      3
Restaurant service absolutely sucks .      3
The service is impeccable .                2
Pool area clean and refreshing .           2
The ambience is wonderfully peaceful .     2
Great desk service , too .                 2
Room decor is outdated .                   2
Horribly rude staff .                      1
The beds are extremely comfortable .       1
Excellent customer service .               1
Front desk staff was inept .               1
Parking is easy .                          1
Wifi was very reliable .                   1
Name: key_point_x, dtype: int64

In [72]:
selected_kps = [
    'Friendly and courteous staff .  ',
    'Clean and comfortable rooms .',
    'Great food selection',
    'Poor service .',
#     'Beds were comfortable .',
    'The food is less consistent .  ',
#     'Not worth the money'
]

In [83]:
# valid_df = valid_df[valid_df['full_key_point'].isin(selected_kps)]

In [84]:
valid_df = valid_df[valid_df['full_key_point'].isin(valid_kp.head(2).index)]

In [85]:
valid_df['full_key_point'].value_counts()

They have excellent customer service .    29
Very professional staff .                 13
Name: full_key_point, dtype: int64

In [86]:
valid_df.shape

(42, 16)

In [89]:
hotel_pred_df = merged_df[merged_df['arg_id'].str.startswith('arg_1')].merge(kp_df, on=['key_point_id'])
# hotel_pred_df = hotel_pred_df[hotel_pred_df['full_key_point'].isin(selected_kps)]
hotel_pred_df = hotel_pred_df[hotel_pred_df['key_point_x'].isin(valid_kp.head(2).index)]
hotel_pred_df['pred_label'] = 1
hotel_pred_df

,arg_id,topic_x,argument,num_of_aspects,aspects_x,opinions_x,opinion_aspect_pairs_x,sentiments_x,attributes_x,key_point_id,...,label_relaxed,topic_y,key_point_y,aspects,opinions,opinion_aspect_pairs,sentiments,attributes,predicted_WA_y,pred_label
0,arg_1_1001_1,Automotive,"Overall , Andrew provides a truly consultative process without any sales pressure .",1,process,truly consultative,truly consultative process,positive,staff,kp_1_2,...,1,Automotive,They have excellent customer service .,customer service,excellent,excellent customer service,positive,staff,0.585227,1
1,arg_1_1102_1,Automotive,She is a very kind and respectful sales person .,1,sales person,very kind and respectful,very kind and respectful sales person,positive,staff,kp_1_2,...,0,Automotive,They have excellent customer service .,customer service,excellent,excellent customer service,positive,staff,0.585227,1
2,arg_1_1454_2,Automotive,Their dedication to resolving this major safety problem was incredible .,2,safety problem,incredible,incredible safety problem,positive,restaurant -> atmosphere,kp_1_2,...,1,Automotive,They have excellent customer service .,customer service,excellent,excellent customer service,positive,staff,0.585227,1
3,arg_1_1520_2,Automotive,They have great offers and awesome customer service too .,2,customer service,awesome,awesome customer service,positive,staff,kp_1_2,...,1,Automotive,They have excellent customer service .,customer service,excellent,excellent customer service,positive,staff,0.585227,1
4,arg_1_1525_1,Automotive,"They have such amazing customer service , it was n't just them",1,customer service,amazing,amazing customer service,positive,staff,kp_1_2,...,1,Automotive,They have excellent customer service .,customer service,excellent,excellent customer service,positive,staff,0.585227,1
5,arg_1_1553_1,Automotive,"They were fast , friendly , & the customer service was superb !",1,customer service,superb,superb customer service,positive,staff,kp_1_2,...,1,Automotive,They have excellent customer service .,customer service,excellent,excellent customer service,positive,staff,0.585227,1
6,arg_1_1652_1,Automotive,Tracie Seib gave us outstanding customer service .,1,customer service,outstanding,outstanding customer service,positive,staff,kp_1_2,...,1,Automotive,They have excellent customer service .,customer service,excellent,excellent customer service,positive,staff,0.585227,1
7,arg_1_1661_1,Automotive,Use the check - in offer on Yelp for a solid discount .,1,discount,solid,solid discount,positive,value-for-money,kp_1_2,...,0,Automotive,They have excellent customer service .,customer service,excellent,excellent customer service,positive,staff,0.585227,1
8,arg_1_361_1,Automotive,Great communication between staff and customers .,1,communication,great,great communication,positive,restaurant -> atmosphere,kp_1_2,...,1,Automotive,They have excellent customer service .,customer service,excellent,excellent customer service,positive,staff,0.585227,1
9,arg_1_95_1,Automotive,Audi has the best customer service & streamlined purchase program .,1,customer service,best,best customer service,positive,staff,kp_1_2,...,1,Automotive,They have excellent customer service .,customer service,excellent,excellent customer service,positive,staff,0.585227,1


In [90]:
hotel_pred_df.shape

(10, 31)

In [91]:
# valid_df = valid_df.merge(hotel_pred_df[['arg_id', 'key_point_id', 'pred_label']], how='left')
# valid_df['pred_label'] = valid_df['pred_label'].fillna(0)
# valid_df

In [92]:
from sklearn.metrics import recall_score, precision_score

new_valid_df = valid_df.merge(hotel_pred_df[['arg_id', 'key_point_id', 'pred_label']], how='right')
new_valid_df['label'] = new_valid_df['label'].fillna(0)
print(precision_score(new_valid_df['label'], new_valid_df['pred_label']))

new_valid_df = valid_df.merge(hotel_pred_df[['arg_id', 'key_point_id', 'pred_label']], how='left')
new_valid_df['pred_label'] = new_valid_df['pred_label'].fillna(0)
print(recall_score(new_valid_df['label'], new_valid_df['pred_label']))

0.8
0.19047619047619047


In [66]:
domain_mapping = {
    'Arts & Entertainment': 0,
    'Automotive': 1,
    'Beauty & Spas': 2,
    'Hotels': 3,
    'Restaurants': 4
}
from sklearn.metrics import recall_score, precision_score

def calculate_precision_recall(merged_df, df, domain, top=None, selected_kps=[]):
    valid_df = df[(df['label'] == 1) & (df['arg_id'].str.startswith(f'arg_{domain_mapping[domain]}'))]
    valid_kp = valid_df['full_key_point'].value_counts()
    
    top_kp = merged_df.drop(['key_point'], axis=1)[merged_df['arg_id'].str.startswith(f'arg_{domain_mapping[domain]}')]\
        .merge(kp_df, on=['key_point_id'])['key_point'].value_counts()
    result = top_kp[top_kp.index.isin(valid_kp.index.tolist())]
    if top != None:
        display(result[result.index.isin(valid_kp.head(top).index)])
    elif len(selected_kps) > 0:
        display(result[result.index.isin(selected_kps)])
    else:
        display(result)
    
    if top != None:
        valid_df = valid_df[valid_df['full_key_point'].isin(valid_kp.head(top).index)]
    elif len(selected_kps) > 0:
        valid_df = valid_df[valid_df['full_key_point'].isin(selected_kps)]
    
    pred_df = merged_df[merged_df['arg_id'].str.startswith(f'arg_{domain_mapping[domain]}')].drop(['key_point'], axis=1)\
        .merge(kp_df, on=['key_point_id'])
    if top != None:
        pred_df = pred_df[pred_df['key_point'].isin(valid_kp.head(top).index)]
    elif len(selected_kps) > 0:
        pred_df = pred_df[pred_df['key_point'].isin(selected_kps)]
    
#     pred_df = pred_df[pred_df['score'] >= 0.5]
#     pred_df = pred_df[pred_df['score'] >= 0.2]
    pred_df['pred_label'] = 1
#     pred_df.loc[pred_df['score'] >= 0.5 ,'pred_label'] = 1
#     pred_df.loc[pred_df['score'] < 0.5 ,'pred_label'] = 0
    
    new_valid_df = valid_df.merge(pred_df[['arg_id', 'key_point_id', 'pred_label']], how='right')
    new_valid_df['label'] = new_valid_df['label'].fillna(0)
    precision = precision_score(new_valid_df['label'], new_valid_df['pred_label'])

    new_valid_df = valid_df.merge(pred_df[['arg_id', 'key_point_id', 'pred_label']], how='left')
    new_valid_df['pred_label'] = new_valid_df['pred_label'].fillna(0)
    recall = recall_score(new_valid_df['label'], new_valid_df['pred_label'])
    
    return precision, recall

In [151]:
# merged_df = merged_df.drop_duplicates(subset=['arg_id'])

In [113]:
# merged_df.set_index("arg_id")["key_point_id"].to_dict()

In [67]:
calculate_precision_recall(merged_df, df, 'Arts & Entertainment', 2)

Friendly and helpful staff .       10
Awesome service and employees .     3
Name: key_point, dtype: int64

(0.9230769230769231, 0.5)

In [68]:
calculate_precision_recall(merged_df, df, 'Arts & Entertainment', 3)

Friendly and helpful staff .       10
Awesome service and employees .     3
It had wonderful acoustics .        1
Name: key_point, dtype: int64

(0.9285714285714286, 0.3939393939393939)

In [69]:
calculate_precision_recall(merged_df, df, 'Arts & Entertainment', None)

Friendly and helpful staff .                10
Superior customer service !                  5
Very talented musicians !                    4
Seats are adequately comfortable .           4
Parking was fast and FREE !                  4
Horrible customer service .                  3
Rude and Unprofessional staff .              3
Awesome service and employees .              3
Incredibly talented performers - wow !       2
Nice employees ... not to pushy .            2
Awesome lighting , and decor .               1
It had wonderful acoustics .                 1
Worth the money .                            1
lines were short and quick .                 1
Terrible administration and management .     1
The acoustics are poor .                     1
Name: key_point, dtype: int64

(0.5172413793103449, 0.43478260869565216)

In [89]:
selected_kps = ['Friendly and helpful staff .', 'Seats are adequately comfortable .   ', 'Horrible customer service .']
calculate_precision_recall(merged_df, df, 'Arts & Entertainment', None, selected_kps)

Friendly and helpful staff .             10
Seats are adequately comfortable .        4
Horrible customer service .               3
Name: key_point, dtype: int64

(0.9411764705882353, 0.7619047619047619)

In [68]:
calculate_precision_recall(merged_df, df, 'Automotive', 2)

They have excellent customer service .    10
Name: key_point, dtype: int64

(0.8, 0.19047619047619047)

In [69]:
calculate_precision_recall(merged_df, df, 'Automotive', 3)

They have excellent customer service .    10
The employees here are wonderful !         2
Name: key_point, dtype: int64

(0.8333333333333334, 0.18181818181818182)

In [70]:
calculate_precision_recall(merged_df, df, 'Automotive', None)

FAST and FRIENDLY SERVICE ! ! !           11
They have excellent customer service .    10
Very courteous staff .                     9
Store was clean and organized .            6
Their work is Impeccable .                 5
The employees here are wonderful !         2
The cashiers are always friendly .         2
Overall just poor quality .                1
Prices were fair and reasonable .          1
TERRIBLE CUSTOMER SERVICE !                1
Extremely poor service .                   1
Name: key_point, dtype: int64

(0.4, 0.23655913978494625)

In [75]:
# selected_kps = ['FAST and FRIENDLY SERVICE ! ! !', 'They have excellent customer service .', 'Store was clean and organized .  ']
# calculate_precision_recall(merged_df, df, 'Automotive', None, selected_kps)

In [76]:
# selected_kps = ['FAST and FRIENDLY SERVICE ! ! !', 'The employees here are wonderful !', 'Store was clean and organized .  ']
# calculate_precision_recall(merged_df, df, 'Automotive', None, selected_kps)

In [77]:
# selected_kps = ['Store was clean and organized .  ', 'The employees here are wonderful !', 'Very professional staff .']
# calculate_precision_recall(merged_df, df, 'Automotive', None, selected_kps)

In [67]:
calculate_precision_recall(merged_df, df, 'Beauty & Spas', 2)

Staff is friendly and accommodating .    13
Amazing & professional service .          3
Name: key_point, dtype: int64

(0.9375, 0.46875)

In [68]:
calculate_precision_recall(merged_df, df, 'Beauty & Spas', 3)

Staff is friendly and accommodating .    13
Customer service- EXCELLENT !             7
Amazing & professional service .          3
Name: key_point, dtype: int64

(0.8695652173913043, 0.4444444444444444)

In [69]:
calculate_precision_recall(merged_df, df, 'Beauty & Spas', None)

Staff is friendly and accommodating .    13
Customer service- EXCELLENT !             7
Super clean and friendly salon .          5
Very professionally run business .        5
Amazing & professional service .          3
Artists are highly skilled .              3
This place is TERRIFIC .                  3
Super comfy and great massage .           2
POOR CUSTOMER SERVICE .                   2
Prices were fair and reasonable .         1
Name: key_point, dtype: int64

(0.5357142857142857, 0.38961038961038963)

In [108]:
selected_kps = ['Staff is friendly and accommodating .', 'Customer service- EXCELLENT !',  'This place is TERRIFIC .  ']
calculate_precision_recall(merged_df, df, 'Beauty & Spas', None, selected_kps)

Staff is friendly and accommodating .    13
Customer service- EXCELLENT !             7
This place is TERRIFIC .                  3
Name: key_point, dtype: int64

(0.8260869565217391, 0.59375)

In [67]:
calculate_precision_recall(merged_df, df, 'Hotels', 2)

Series([], Name: key_point, dtype: int64)

UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.

In [68]:
calculate_precision_recall(merged_df, df, 'Hotels', 3)

Friendly and helpful staff .     16
Clean and comfortable rooms .    12
Excellent customer service .      1
Name: key_point, dtype: int64

(0.9655172413793104, 0.6829268292682927)

In [69]:
calculate_precision_recall(merged_df, df, 'Hotels', None)

Friendly and helpful staff .              16
Clean and comfortable rooms .             12
Very peaceful and relaxing location .      3
Restaurant service absolutely sucks .      3
The service is impeccable .                2
Pool area clean and refreshing .           2
The ambience is wonderfully peaceful .     2
Great desk service , too .                 2
Room decor is outdated .                   2
Horribly rude staff .                      1
The beds are extremely comfortable .       1
Excellent customer service .               1
Front desk staff was inept .               1
Parking is easy .                          1
Wifi was very reliable .                   1
Name: key_point, dtype: int64

(0.6724137931034483, 0.5416666666666666)

In [70]:
# sorted_kp_id = valid_df['key_point_id'].value_counts().index
# # print(sorted_kp_id)
# # new_df = df[df['isMultiAspect'] == False]
# new_df = df
# new_df = new_df.set_index(['key_point_id']).loc[sorted_kp_id].reset_index().rename(columns={'index': 'key_point_id'})
# # new_df = new_df.drop_duplicates(subset='arg_id')
# new_df = new_df.groupby(['arg_id']).apply(lambda grp: grp.head(len(grp['attributes'].iloc[0]))).reset_index(drop=True)
# new_df

In [71]:
selected_kps = ['Friendly and helpful staff .', 'Clean and comfortable rooms .', 
                'The service is impeccable .']
calculate_precision_recall(merged_df, df, 'Hotels', None, selected_kps)

Friendly and helpful staff .     16
Clean and comfortable rooms .    12
The service is impeccable .       2
Name: key_point, dtype: int64

(0.9333333333333333, 0.717948717948718)

In [72]:
selected_kps = ['Friendly and helpful staff .', 'Clean and comfortable rooms .', 'The ambience is wonderfully peaceful .']
calculate_precision_recall(merged_df, df, 'Hotels', None, selected_kps)

Friendly and helpful staff .              16
Clean and comfortable rooms .             12
The ambience is wonderfully peaceful .     2
Name: key_point, dtype: int64

(0.9333333333333333, 0.8)

In [68]:
calculate_precision_recall(merged_df, df, 'Restaurants', 2)

Staff was courteous and accommodating .    11
Customer Service is Excellent .             7
Name: key_point, dtype: int64

(0.9444444444444444, 0.5151515151515151)

In [69]:
calculate_precision_recall(merged_df, df, 'Restaurants', 3)

Staff was courteous and accommodating .    11
Customer Service is Excellent .             7
Name: key_point, dtype: int64

(0.9444444444444444, 0.3617021276595745)

In [70]:
calculate_precision_recall(merged_df, df, 'Restaurants', None)

Staff was courteous and accommodating .       11
Fresh food , using local produce .             8
Customer Service is Excellent .                7
The service here was exceptional .             5
Lots of outdoor seating and                    3
AMAZING AUTHENTIC FLAVOR !                     3
Way over priced .                              2
Fantastic drink selection .                    2
Great beer and wine selection .                2
The quality has diminished significantly .     2
Great variety of menu items .                  2
Generous sized portions .                      2
Prices are fair and reasonable .               1
Plenty of coffees and teas .                   1
Name: key_point, dtype: int64

(0.484375, 0.28703703703703703)

In [80]:
# selected_kps = ['Staff was courteous and accommodating .', 
#                 'Fresh food , using local produce .', 'The service here was exceptional .  ']
selected_kps = ['Staff was courteous and accommodating .', 
                'Fresh food , using local produce .', 'Generous sized portions .']
calculate_precision_recall(merged_df, df, 'Restaurants', None, selected_kps)

Staff was courteous and accommodating .    11
Fresh food , using local produce .          8
Generous sized portions .                   2
Name: key_point, dtype: int64

(0.8095238095238095, 0.6296296296296297)